### Importation des librairies

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import plotly

### Lecture des données et convertion des dates

In [2]:
df = pd.read_csv('./data/collisions_routieres.csv')
df["DT_ACCDN"] = pd.to_datetime(df["DT_ACCDN"])
df["year"] = df["DT_ACCDN"].dt.year
df["month"] = df["DT_ACCDN"].dt.month
df["day"] = df["DT_ACCDN"].dt.day
df["weekday"] = df["DT_ACCDN"].dt.weekday

FileNotFoundError: [Errno 2] No such file or directory: '/data/collisions_routieres.csv'

### preparation des données utilisées pour la viz

In [ ]:
def convert_gravite(x, split_mort_grave=False):
    # Takes GRAVITE as input
    if split_mort_grave:
        if x == 'Mortel':
            return 'MORTEL'
        elif x == 'Grave':
            return 'GRAVE'
        else:
            return 'LEGER_MATERIEL'
    else:
        
        if x in ['Mortel', 'Grave']:
            return 'GRAVE_MORTEL'
        else:
            return 'LEGER_MATERIEL'

    
def convert_type(x):
    #Takes nb_camionLourd_tractRoutier as input
    if x > 0:
        return 'CAMION'
    else:
        return 'SANS_CAMION'

In [ ]:
# GRAVE_MORTEL OU LEGER_MATERIEL
df['GRAVITE_RENAME'] = df.loc[:, 'GRAVITE'].apply(lambda x : convert_gravite(x))
df['GRAVITE_SPLIT'] = df.loc[:, 'GRAVITE'].apply(lambda x : convert_gravite(x, split_mort_grave=True))

In [ ]:
# CAMION OU SANS_CAMION
df['TYPE_VEHICULE'] = df.loc[:, 'nb_camionLourd_tractRoutier'].apply(lambda x: convert_type(x))

# MOKCUP

## Viz #1 : Est-ce que les accidents semblent diminuer à Montréal depuis l'annonce de Vision Zéro

#### Version nombre en fonction du temps

In [ ]:
df1_count = df.groupby('DT_ACCDN').size()
df1_count.name = 'count'
df1_count = df1_count.rolling(7, min_periods=1).mean()

In [ ]:
plotly.offline.init_notebook_mode()

px.line(df1_count.reset_index(), x='DT_ACCDN', y='count')

### EXPORT  

In [ ]:
df1_count.reset_index().rename(columns={'DT_ACCDN':'Date', 'count':'Nombres'}).to_csv('rolling7_viz1_all_vehicule_date.csv', index=False)

## Viz #2 : Est-ce que les accidents semblent diminuer à Montréal depuis l'annonce de Vision Zéro pour tous les types de véhicule

In [ ]:
df2_count = df.groupby(['TYPE_VEHICULE','DT_ACCDN']).size()
df2_count.name = 'count'
df2_count = df2_count.rolling(7, min_periods=1).mean()

In [ ]:
plotly.offline.init_notebook_mode()
px.area(df2_count.reset_index(), x='DT_ACCDN', y='count', color='TYPE_VEHICULE', )

### EXPORT

In [ ]:
df2_count.reset_index().rename(columns={'TYPE_VEHICULE':'Types', 'DT_ACCDN':'Date', 'count':'Nombres'}).to_csv('rolling7_viz2_acc_by_type_date.csv', index=False)

In [ ]:
plotly.offline.init_notebook_mode()
px.area(df2_count.loc['CAMION'].reset_index(), x='DT_ACCDN', y='count' )

## export date-camion_count

In [ ]:
df2_count.loc['CAMION'].reset_index().rename(columns={'DT_ACCDN':'Date', 'count':'Nombres'}).to_csv('rolling7_viz3_acc_camion_date.csv', index=False)

## VIZ 3 : Quel est le ratio d'accident GM avec camion par rapport à lorsque les camions sont impliqués?

In [ ]:
df3 = df.groupby(['TYPE_VEHICULE', 'GRAVITE_SPLIT']).size()
df3.name = 'count'
df3 = df3.reset_index()
df3['pourcentage'] = df3.apply(lambda row : row.loc['count'] *100 / df3[df3['TYPE_VEHICULE'] == row.loc['TYPE_VEHICULE']]['count'].sum(),
                         axis=1)

In [ ]:
df3 \
    .pivot(index='TYPE_VEHICULE', columns='GRAVITE_SPLIT', values='count') \
    .reindex(['LEGER_MATERIEL', 'GRAVE', 'MORTEL'], axis=1) \
    .to_csv('stackbar_count.csv')

In [ ]:
df3.to_csv('barchart_ratio.csv', index='False')

In [ ]:
df3[df3.GRAVITE_SPLIT != 'LEGER_MATERIEL']

In [ ]:
plotly.offline.init_notebook_mode()
px.bar(df3, x='TYPE_VEHICULE', y='pourcentage', color='GRAVITE_SPLIT', hover_data=['count'])

## VIZ 3 : CONTEXTE DES ACCIDENTS (

In [ ]:
# Garder seulement les 20k accidents de camion et comprendre mieux les 174 qui sont graves ou mortel
# pour CD_ENVRN_ACCDN, CD_CATEG_ROUTE, VITESSE_AUTOR

In [ ]:
def viz_context(data_df, var):
    col = ['TYPE_VEHICULE', 'GRAVITE_RENAME']
    col.append(var)
    df4 = data_df.groupby(col).size().loc['CAMION']
    df4.name = 'nombre'
    df4 = df4.reset_index()
    df4['pourcentage'] = df4.apply(lambda row : row.loc['nombre']/ df4[df4['GRAVITE_RENAME'] == row['GRAVITE_RENAME']]['nombre'].sum(),
                                   axis=1)
    fig = px.bar(df4,
        x=var,
        y='pourcentage',
        color='GRAVITE_RENAME',
        barmode='group')
    return fig


In [ ]:
plotly.offline.init_notebook_mode()
viz_context(df, 'CD_ENVRN_ACCDN')

In [ ]:
plotly.offline.init_notebook_mode()
viz_context(df, 'CD_CATEG_ROUTE')

In [ ]:
plotly.offline.init_notebook_mode()
viz_context(df, 'VITESSE_AUTOR')

### TEST PARRALLE LEGISLATIF

In [ ]:
plotly.offline.init_notebook_mode()
col5 = ['TYPE_VEHICULE', 'GRAVITE_RENAME', 'CD_ENVRN_ACCDN', 'CD_CATEG_ROUTE', 'VITESSE_AUTOR']
df5 = df.groupby(col5).size()
df5.name = 'count'
df5 = df5.loc['CAMION']
px.parallel_categories(df5.reset_index())

### EXPORT

#### Rename before export

In [ ]:
df5_export = df5.reset_index().rename(columns={"GRAVITE_RENAME":"Gravite",
                                  "CD_ENVRN_ACCDN": "Environnement",
                                  "CD_CATEG_ROUTE" : "Route",
                                  "VITESSE_AUTOR" : "Vitesse"})


df5_export['Gravite'] = df5_export.loc[:,'Gravite'].replace({
    'GRAVE_MORTEL': 'Grave ou Mortel',
    'LEGER_MATERIEL' : 'Dommage matériel ou Léger'})


df5_export['Environnement'] = df5_export.loc[:,'Environnement'].replace({1.0:'Scolaire',
                                                                        2.0: 'Residentiel',
                                                                        3.0: 'Commercial',
                                                                        4.0: 'Industriel',
                                                                        5.0: 'Rural',
                                                                        6.0: 'Forestier',
                                                                        7.0: 'Recreatif',
                                                                        9.0: 'Autre',
                                                                        0.0: 'ND'})

df5_export['Route'] = df5_export.loc[:,'Route'].replace({11.0:'Chemin public: bretelle/collecteur d’autoroute/voie de service',
                                                         12.0: 'Chemin public: route numérotée',
                                                         13.0: 'Chemin public: artère principale',
                                                         14.0: 'Chemin public: rue résidentielle',
                                                         15.0: 'Chemin public: chemin/rang',
                                                         16.0: 'Chemin public: ruelle',
                                                         19.0: 'Chemin public: autre',
                                                         21.0: 'Hors chemin public: terrain de stationnement',
                                                         22.0: 'Hors chemin public: terrain privé',
                                                         23.0: 'Hors chemin public: chemin privé',
                                                         24.0: 'Hors chemin public: chemin forestier',
                                                         25.0: 'Hors chemin public: sentier balisé',
                                                         29.0: 'Hors chemin public: autre'})

df5_export['Vitesse'] = df5_export.loc[:,'Vitesse'].apply(lambda x : str(int(x)) + ' Km/h')


In [ ]:
df5_export.head()

In [ ]:
df5_export.to_csv('pset_env_route_vit.csv', index=False)

In [ ]:
plotly.offline.init_notebook_mode()
col6 = ['TYPE_VEHICULE', 'GRAVITE_RENAME', 'CD_CONFG_ROUTE', 'CD_ASPCT_ROUTE']
df6 = df.groupby(col6).size()
df6.name = 'count'
df6pp = df6.loc['CAMION'].loc['GRAVE_MORTEL'].reset_index()
px.parallel_categories(df6pp.iloc[:, 0:3])

### Export viz 6

In [ ]:
df6_export = df6.reset_index().rename(columns={"GRAVITE_RENAME":"Gravite",
                                  "CD_CONFG_ROUTE": "Configuration",
                                  "CD_ASPCT_ROUTE" : "Aspect route"})


df6_export['Gravite'] = df6_export.loc[:,'Gravite'].replace({
    'GRAVE_MORTEL': 'Grave ou Mortel',
    'LEGER_MATERIEL' : 'Dommage matériel ou Léger'})


df6_export['Configuration'] = df6_export.loc[:,'Configuration'].replace({1.0:'Sens unique',
                                                                        2.0: 'une voie par direction',
                                                                        3.0: 'plus d’une voie par direction',
                                                                        4.0: 'Séparée par aménagement franchissable',
                                                                        5.0: 'Séparée par aménagement infranchissable',
                                                                        9.0: 'Autre'
                                                                        })

df6_export['Aspect route'] = df6_export.loc[:,'Aspect route'].replace({11.0:'Droit, plat',
                                                         12.0: 'Droit, en haut de la pente',
                                                         13.0: 'Droit, dans la pente',
                                                         14.0: 'Droit, en bas de la pente (creux)',
                                                         21.0: 'Courbe, plat',
                                                         22.0: 'Courbe, en haut de la pente',
                                                         23.0: 'Courbe, dans la pente',
                                                         24.0: 'Courbe, en bas de la pente (creux)'})

In [ ]:
df6_export[df6_export['TYPE_VEHICULE'] == 'CAMION'].iloc[:,1:].to_csv('pset_conf_aspect_vit.csv', index=False)

In [ ]:
plotly.offline.init_notebook_mode()
col7 = ['TYPE_VEHICULE', 'GRAVITE_RENAME', 'CD_COND_METEO', 'CD_ETAT_SURFC', 'CD_ETAT_CHASS', 'CD_ECLRM']
df7 = df.groupby(col7).size()
df7.name = 'count'
df7pp = df7.loc['CAMION'].loc['GRAVE_MORTEL'].reset_index()
px.parallel_categories(df7pp.iloc[:,:-1])

In [ ]:
df7_export = df7.reset_index().rename(columns={"GRAVITE_RENAME":"Gravite",
                                  "CD_COND_METEO": "Meteo",
                                  "CD_ETAT_SURFC" : "Etat Surface",
                                  "CD_ETAT_CHASS" : "Etat Chaussee",
                                "CD_ECLRM" : "Eclairage"})


df7_export['Gravite'] = df7_export.loc[:,'Gravite'].replace({
    'GRAVE_MORTEL': 'Grave ou Mortel',
    'LEGER_MATERIEL' : 'Dommage matériel ou Léger'})


df7_export['Meteo'] = df7_export.loc[:,'Meteo'].replace({11.0:'Clair',
                                                         12.0: 'Couvert (nuageux/sombre)',
                                                         13.0: 'Brouillard/brume',
                                                         14.0: 'Pluie/bruine',
                                                         15.0: 'Averse (pluie forte)',
                                                         16.0: 'Vent fort (pas de poudrerie, pas de pluie)',
                                                         17.0: 'Neige/grêle',
                                                         18.0: 'Poudrerie/tempête de neige',
                                                         19.0: 'Verglas',
                                                         99.0: 'Autre'})

df7_export['Etat Chaussee'] = df7_export.loc[:,'Etat Chaussee'].replace({1:'En bon état',
                                                         2: 'En construction/en réparation',
                                                         3: 'Ornières/affaissement',
                                                         4: 'Fissures importantes',
                                                         5: 'Trous/nids-de-poule/cahots',
                                                         6: 'Dénivellation',
                                                         9: 'Autre',
                                                         0: 'Non précisé'})

df7_export['Etat Surface'] = df7_export.loc[:,'Etat Surface'].replace({11.0:'Sèche',
                                                         12.0: 'Mouillée',
                                                         13.0: 'Accumulation d’eau (aquaplanage)',
                                                         14.0: 'Sable, gravier sur la chaussée',
                                                         15.0: 'Gadoue/neige fondante',
                                                         16.0: 'Enneigée',
                                                         17.0: 'Neige durcie',
                                                         18.0: 'Glacée',
                                                         19.0: 'Boueuse',
                                                         20.0: 'Huileuse',
                                                         99.0: 'Autre'})

df7_export['Eclairage'] = df7_export.loc[:,'Eclairage'].replace({1:'Jour et demi-obscurité',
                                                         2: 'Jour et demi-obscurité',
                                                         3: 'Nuit et chemin éclairé',
                                                         4: 'Nuit et chemin non éclairé'})

In [ ]:
df7_export[df7_export['TYPE_VEHICULE'] == 'CAMION'].iloc[:,1:].to_csv('pset_cond_ext.csv', index=False)

In [ ]:
df7_export['Etat Surface'].value_counts()